# Tracking electoral con ponderación censal 2022

Notebook de ejemplo para seguimiento de imagen e intención de voto con ponderación basada en el Censo 2022 (por provincia, sexo y grupo de edad).

## Configuración y dependencias

In [ ]:
import unicodedata
from pathlib import Path
from typing import Dict, Iterable, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)


## Catálogo de provincias (códigos oficiales)

El diccionario permite imputar y completar nombres parciales (ej. `cord` → `CORDOBA`).

In [ ]:
PROVINCIAS: Dict[str, str] = {
    "01": "BUENOS AIRES",
    "02": "CABA",
    "03": "CATAMARCA",
    "04": "CHACO",
    "05": "CHUBUT",
    "06": "CORDOBA",
    "07": "CORRIENTES",
    "08": "ENTRE RIOS",
    "09": "FORMOSA",
    "10": "JUJUY",
    "11": "LA PAMPA",
    "12": "LA RIOJA",
    "13": "MENDOZA",
    "14": "MISIONES",
    "15": "NEUQUEN",
    "16": "RIO NEGRO",
    "17": "SALTA",
    "18": "SAN JUAN",
    "19": "SAN LUIS",
    "20": "SANTA CRUZ",
    "21": "SANTA FE",
    "22": "SANTIAGO DEL ESTERO",
    "23": "TIERRA DEL FUEGO",
    "24": "TUCUMAN",
}
PROVINCIAS_NORMALIZADAS = {v.lower(): k for k, v in PROVINCIAS.items()}


## Funciones de normalización
- Provincias: intenta mapear por código, nombre completo o fragmentos sin tildes.
- Sexo: interpreta prefijos `fe` o `mu` como femenino.
- Edad: se imputan faltantes con la mediana, se redondea a entero y se descartan casos menores de 16 o mayores de 95 (se marcan como 999 antes de eliminar).

In [ ]:
def quitar_tildes(texto: str) -> str:
    return "".join(c for c in unicodedata.normalize("NFD", texto) if unicodedata.category(c) != "Mn")


def normalizar_provincia(valor: str) -> Tuple[str, str]:
    if pd.isna(valor):
        return None, None
    limpio = quitar_tildes(str(valor)).strip().lower()
    if not limpio:
        return None, None

    # Coincidencia por código explícito
    if limpio.zfill(2) in PROVINCIAS:
        codigo = limpio.zfill(2)
        return codigo, PROVINCIAS[codigo]

    # Coincidencia exacta por nombre
    if limpio in PROVINCIAS_NORMALIZADAS:
        codigo = PROVINCIAS_NORMALIZADAS[limpio]
        return codigo, PROVINCIAS[codigo]

    # Coincidencia por fragmento
    for nombre, codigo in PROVINCIAS_NORMALIZADAS.items():
        if limpio in nombre or nombre.startswith(limpio):
            return codigo, PROVINCIAS[codigo]

    return None, None


def normalizar_sexo(valor: str) -> str:
    if pd.isna(valor):
        return None
    limpio = quitar_tildes(str(valor)).strip().lower()
    if not limpio:
        return None
    if limpio.startswith(("fe", "mu")):
        return "Femenino"
    if limpio.startswith(("ma", "ho", "va")):
        return "Masculino"
    return None


def limpiar_edad(serie: pd.Series) -> pd.Series:
    numeros = pd.to_numeric(serie, errors="coerce")
    mediana = numeros.median()
    imputada = numeros.fillna(mediana).round().astype(int)
    fuera_de_rango = (imputada < 16) | (imputada > 95)
    imputada.loc[fuera_de_rango] = 999
    return imputada


def agrupar_edad(edad: int) -> str:
    if pd.isna(edad) or edad < 18 or edad == 999:
        return None
    if edad < 30:
        return "18-29"
    if edad < 45:
        return "30-44"
    if edad < 60:
        return "45-59"
    return "60+"


## Carga de censos y encuestas
La distribución censal se calcula a partir del CSV (`poblacion` → `prop_objetivo`).

In [ ]:
DATA_DIR = Path("data")
OUTPUT_DIR = Path("outputs")
OUTPUT_DIR.mkdir(exist_ok=True)


def cargar_censo(path: Path) -> pd.DataFrame:
    censo = pd.read_csv(path)
    total = censo["poblacion"].sum()
    censo["prop_objetivo"] = censo["poblacion"] / total
    return censo


def cargar_encuestas(ruta: Path) -> pd.DataFrame:
    archivos = sorted(ruta.glob("encuestas_*.csv"))
    if not archivos:
        raise FileNotFoundError("No se encontraron archivos encuestas_*.csv en el directorio de datos")
    frames = []
    for archivo in archivos:
        df = pd.read_csv(archivo, parse_dates=["Fecha"])
        df["archivo"] = archivo.name
        frames.append(df)
    return pd.concat(frames, ignore_index=True)


## Limpieza y ponderación
- Provincias traducidas a código + nombre.
- Sexo inferido por prefijos.
- Edades imputadas y filtradas.
- Pesos calculados contra la distribución censal cargada dinámicamente.

In [ ]:
def limpiar_encuestas(df: pd.DataFrame) -> pd.DataFrame:
    base = df.copy()
    base["Edad"] = limpiar_edad(base["Edad"])
    base = base[base["Edad"] != 999]

    provincia_info = base["Estrato"].apply(normalizar_provincia)
    base["codigo_provincia"] = provincia_info.apply(lambda x: x[0])
    base["Provincia"] = provincia_info.apply(lambda x: x[1])

    base["Sexo"] = base["Sexo"].apply(normalizar_sexo)
    base["grupo_edad"] = base["Edad"].apply(agrupar_edad)

    base["Voto"] = base["Voto"].astype(str).str.strip().str.title()
    base["Voto Anterior"] = base["Voto Anterior"].astype(str).str.strip().str.title()
    base["Imagen del Candidato"] = pd.to_numeric(base["Imagen del Candidato"], errors="coerce")

    base = base.dropna(subset=["codigo_provincia", "Provincia", "Sexo", "grupo_edad", "Imagen del Candidato", "Voto"])
    return base


def calcular_pesos(encuestas: pd.DataFrame, censo: pd.DataFrame) -> pd.DataFrame:
    claves = ["codigo_provincia", "Provincia", "Sexo", "grupo_edad"]
    muestra = encuestas.groupby(claves).size().reset_index(name="n_muestra")
    muestra["prop_muestra"] = muestra["n_muestra"] / muestra["n_muestra"].sum()

    objetivo = censo.rename(columns={"sexo": "Sexo", "grupo_edad": "grupo_edad", "codigo_provincia": "codigo_provincia"})
    mergeado = objetivo.merge(muestra, how="left", on=["codigo_provincia", "Sexo", "grupo_edad"])
    mergeado["prop_muestra"].fillna(0, inplace=True)
    mergeado["peso"] = mergeado.apply(
        lambda fila: fila["prop_objetivo"] / fila["prop_muestra"] if fila["prop_muestra"] > 0 else 0,
        axis=1,
    )

    pesos = mergeado[["codigo_provincia", "Sexo", "grupo_edad", "peso"]]
    return encuestas.merge(pesos, how="left", on=["codigo_provincia", "Sexo", "grupo_edad"])


## Ejecución del pipeline

In [ ]:
censo = cargar_censo(DATA_DIR / "censo_2022_distribucion.csv")
encuestas_raw = cargar_encuestas(DATA_DIR)
encuestas_limpias = limpiar_encuestas(encuestas_raw)
encuestas_pesadas = calcular_pesos(encuestas_limpias, censo)
encuestas_pesadas.head()


## Métricas diarias (promedios ponderados sin suavizado)

In [ ]:
def media_ponderada(df: pd.DataFrame, valor_col: str, peso_col: str = "peso") -> float:
    pesos = df[peso_col].fillna(0)
    if pesos.sum() == 0:
        return float("nan")
    return (df[valor_col] * pesos).sum() / pesos.sum()


def evolucion_imagen(df: pd.DataFrame, candidato_objetivo: str) -> pd.DataFrame:
    filtro = df[df["Voto"] == candidato_objetivo]
    return (
        filtro.groupby("Fecha")
        .apply(lambda g: media_ponderada(g, "Imagen del Candidato"))
        .reset_index(name="Imagen ponderada")
        .sort_values("Fecha")
    )


def evolucion_intencion(df: pd.DataFrame, candidato_objetivo: str) -> pd.DataFrame:
    df = df.copy()
    df["intencion_binaria"] = (df["Voto"] == candidato_objetivo).astype(int)
    return (
        df.groupby("Fecha")
        .apply(lambda g: media_ponderada(g, "intencion_binaria"))
        .reset_index(name="Intención de voto")
        .sort_values("Fecha")
    )


## Gráficos (una sola serie por métrica)

In [ ]:
CANDIDATO_OBJETIVO = "Candidato A"

imagen_diaria = evolucion_imagen(encuestas_pesadas, CANDIDATO_OBJETIVO)
intencion_diaria = evolucion_intencion(encuestas_pesadas, CANDIDATO_OBJETIVO)

fig, ax = plt.subplots()
sns.lineplot(data=imagen_diaria, x="Fecha", y="Imagen ponderada", marker="o", ax=ax)
ax.set_title(f"Evolución de imagen - {CANDIDATO_OBJETIVO}")
ax.set_ylabel("Imagen ponderada")
fig.autofmt_xdate()
plt.savefig(OUTPUT_DIR / "imagen_ponderada.png", bbox_inches="tight")
plt.show()

fig, ax = plt.subplots()
sns.lineplot(data=intencion_diaria, x="Fecha", y="Intención de voto", marker="o", ax=ax)
ax.set_title(f"Intención de voto - {CANDIDATO_OBJETIVO}")
ax.set_ylabel("Proporción ponderada")
fig.autofmt_xdate()
plt.savefig(OUTPUT_DIR / "intencion_ponderada.png", bbox_inches="tight")
plt.show()


## Tests de hipótesis y regresiones
Modelos simples por variable sociodemográfica para imagen (OLS) e intención de voto (logística binaria).

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats

# Preparación de dataset binario de voto
encuestas_pesadas["voto_objetivo"] = (encuestas_pesadas["Voto"] == CANDIDATO_OBJETIVO).astype(int)

# Regresión lineal de imagen
modelo_ols = smf.wls(
    "Q('Imagen del Candidato') ~ C(Sexo) + C(grupo_edad) + C(Provincia)",
    data=encuestas_pesadas,
    weights=encuestas_pesadas["peso"],
).fit()
print(modelo_ols.summary())

# Regresión logística de intención de voto
modelo_logit = smf.glm(
    "voto_objetivo ~ C(Sexo) + C(grupo_edad) + C(Provincia)",
    data=encuestas_pesadas,
    family=sm.families.Binomial(),
    freq_weights=encuestas_pesadas["peso"],
).fit()
print(modelo_logit.summary())

# Test de hipótesis (ejemplo: diferencia de imagen por sexo)
imagen_f = encuestas_pesadas.loc[encuestas_pesadas["Sexo"] == "Femenino", "Imagen del Candidato"]
imagen_m = encuestas_pesadas.loc[encuestas_pesadas["Sexo"] == "Masculino", "Imagen del Candidato"]
t_stat, p_val = stats.ttest_ind(imagen_f, imagen_m, equal_var=False)
print(f"T-test imagen F vs M: t={t_stat:.3f}, p={p_val:.4f}")
